In [2]:
pip install nltk

  Using cached nltk-3.8.1-py3-none-any.whl (1.5 MB)
     -------------------------------------- 268.1/268.1 kB 1.4 MB/s eta 0:00:00

[notice] A new release of pip available: 22.1.2 -> 23.1.2
[notice] To update, run: C:\Users\canri\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\canri\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [2]:
def _create_frequency_matrix(sentences):
    frequency_matrix = {}
    stopWords = set(stopwords.words("english"))
    ps = PorterStemmer()

    for sent in sentences:
        freq_table = {}
        words = word_tokenize(sent)
        for word in words:
            word = word.lower()
            word = ps.stem(word)
            if word in stopWords:
                continue

            if word in freq_table:
                freq_table[word] += 1
            else:
                freq_table[word] = 1

        frequency_matrix[sent[:15]] = freq_table

    return frequency_matrix

def _create_tf_matrix(freq_matrix):
    tf_matrix = {}

    for sent, f_table in freq_matrix.items():
        tf_table = {}

        count_words_in_sentence = len(f_table)
        for word, count in f_table.items():
            tf_table[word] = count / count_words_in_sentence

        tf_matrix[sent] = tf_table

    return tf_matrix

def _create_documents_per_words(freq_matrix):
    word_per_doc_table = {}

    for sent, f_table in freq_matrix.items():
        for word, count in f_table.items():
            if word in word_per_doc_table:
                word_per_doc_table[word] += 1
            else:
                word_per_doc_table[word] = 1

    return word_per_doc_table


def _create_documents_per_words(freq_matrix):
    word_per_doc_table = {}

    for sent, f_table in freq_matrix.items():
        for word, count in f_table.items():
            if word in word_per_doc_table:
                word_per_doc_table[word] += 1
            else:
                word_per_doc_table[word] = 1

    return word_per_doc_table



def _create_tf_idf_matrix(tf_matrix, idf_matrix):
    tf_idf_matrix = {}

    for (sent1, f_table1), (sent2, f_table2) in zip(tf_matrix.items(), idf_matrix.items()):

        tf_idf_table = {}

        for (word1, value1), (word2, value2) in zip(f_table1.items(),
                                                    f_table2.items()):  # here, keys are the same in both the table
            tf_idf_table[word1] = float(value1 * value2)

        tf_idf_matrix[sent1] = tf_idf_table

    return tf_idf_matrix


def _score_sentences(tf_idf_matrix) -> dict:
    """
    score a sentence by its word's TF
    Basic algorithm: adding the TF frequency of every non-stop word in a sentence divided by total no of words in a sentence.
    :rtype: dict
    """

    sentenceValue = {}

    for sent, f_table in tf_idf_matrix.items():
        total_score_per_sentence = 0

        count_words_in_sentence = len(f_table)
        for word, score in f_table.items():
            total_score_per_sentence += score

        sentenceValue[sent] = total_score_per_sentence / count_words_in_sentence

    return sentenceValue


def _find_average_score(sentenceValue) -> int:
    """
    Find the average score from the sentence value dictionary
    :rtype: int
    """
    sumValues = 0
    for entry in sentenceValue:
        sumValues += sentenceValue[entry]

    # Average value of a sentence from original summary_text
    average = (sumValues / len(sentenceValue))

    return average

def _generate_summary(sentences, sentenceValue, threshold):
    sentence_count = 0
    summary = ''

    for sentence in sentences:
        if sentence[:15] in sentenceValue and sentenceValue[sentence[:15]] >= (threshold):
            summary += " " + sentence
            sentence_count += 1

    return summary

def _create_idf_matrix(freq_matrix, count_doc_per_words, total_documents):
    idf_matrix = {}

    for sent, f_table in freq_matrix.items():
        idf_table = {}

        for word in f_table.keys():
            idf_table[word] = math.log10(total_documents / float(count_doc_per_words[word]))

        idf_matrix[sent] = idf_table

    return idf_matrix
#sentences = sent_tokenize(text) # NLTK function
#total_documents = len(sentences)

In [3]:
text = article

In [4]:
import math

from nltk import sent_tokenize, word_tokenize, PorterStemmer
from nltk.corpus import stopwords

sentences = sent_tokenize(text)
total_documents = len(sentences)
freq_matrix = _create_frequency_matrix(sentences)
tf_matrix = _create_tf_matrix(freq_matrix)
count_doc_per_words = _create_documents_per_words(freq_matrix)
idf_matrix = _create_idf_matrix(freq_matrix, count_doc_per_words, total_documents)
tf_idf_matrix = _create_tf_idf_matrix(tf_matrix, idf_matrix)
sentence_scores = _score_sentences(tf_idf_matrix)
threshold = _find_average_score(sentence_scores)


In [5]:
summary = _generate_summary(sentences, sentence_scores, 1.5 * threshold) #1.3
print(summary)

 Fans should also consider the possibility that Cawthon is just trolling in an attempt to throw them off the scent of an early release.


In [6]:
import nltk
from nltk.translate.bleu_score import sentence_bleu, corpus_bleu
#nltk.download('punkt')

def calculate_bleu_score(candidate, reference):
    candidate_tokens = nltk.word_tokenize(candidate)
    reference_tokens = nltk.word_tokenize(reference)

    # Calculate the BLEU score
    bleu_score = sentence_bleu([reference_tokens], candidate_tokens)

    return bleu_score


def calculate_corpus_bleu_score(candidates, reference):
    candidates_tokens = [nltk.word_tokenize(candidate) for candidate in candidates]
    reference_tokens = [nltk.word_tokenize(reference)]

    # Calculate the BLEU score
    bleu_score = corpus_bleu([reference_tokens], candidates_tokens)

    return bleu_score




In [7]:
candidate_summaries = [
    summary,
]

In [8]:
validation_truth = "some of the plot elements are so disturbing that they are making him feel sick"

for candidate in candidate_summaries:
    score = calculate_bleu_score(candidate, validation_truth)
    print("BLEU Score:", score)

corpus_score = calculate_corpus_bleu_score(candidate_summaries, validation_truth)
print("Corpus BLEU Score:", corpus_score)

BLEU Score: 1.072268715173723e-231
Corpus BLEU Score: 1.072268715173723e-231


C:\Users\canri\AppData\Local\Programs\Python\Python39\lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
C:\Users\canri\AppData\Local\Programs\Python\Python39\lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
C:\Users\canri\AppData\Local\Programs\Python\Python39\lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower o

In [9]:
import json

In [18]:
with open('validation_changed.json', 'r') as json_file:
    json_list = list(json_file)

val_set = []
targetParagraphs = []
spoilers = []
for json_str in json_list:
    result = json.loads(json_str)
    #val_set.append(result)
    targetParagraphs.append(result['targetParagraphs'])
    spoilers.append(result['spoiler'])

In [20]:
#targetParagraphs

In [33]:
#tf_idf_spoilers

In [36]:

import pickle


with open('my_list.pkl', 'wb') as f:
    pickle.dump(tf_idf_spoilers, f)

In [37]:
tf_idf_spoilers

[(' Source: Scott Cawthon',
  'some of the plot elements are so disturbing that they are making him feel sick'),
 (' In the meantime, the judge had unusually harsh words for Arpaio.??? He ordered the sheriff to put an end to the practice.??? Snow declined.??? Longtime opponents immediately called for the sheriff to resign.???',
  '"intentionally" could transform a court case against Phoenix-area Sheriff Joe Arpaio from civil charges to a criminal prosecution'),
 (' Is there such a thing as too much? And what if you got a discount on the service?',
  '20%'),
 (" What are you up to today???? Are you more of a theater guy or a film guy???? Just the thought of it scares me. It was incredible.??? ... He plays the cab driver. Did he give you any notes while filming???? Were you comfortable with that? How did that go down???? But I enjoy that side of things.??? What did you think of the band you were portraying???? I listened to all their songs completely. ... I love Alan. I like Michael Gamb

In [ ]:
scorelist = []
tf_idf_spoilers = []
for idx, article in enumerate(targetParagraphs):
    sentences = sent_tokenize(article)
    total_documents = len(sentences)
    freq_matrix = _create_frequency_matrix(sentences)
    tf_matrix = _create_tf_matrix(freq_matrix)
    count_doc_per_words = _create_documents_per_words(freq_matrix)
    idf_matrix = _create_idf_matrix(freq_matrix, count_doc_per_words, total_documents)
    tf_idf_matrix = _create_tf_idf_matrix(tf_matrix, idf_matrix)
    sentence_scores = _score_sentences(tf_idf_matrix)
    threshold = _find_average_score(sentence_scores)

    spoiler = _generate_summary(sentences, sentence_scores, 1.5 * threshold) #1.3
    truth = spoilers[idx]
    score = calculate_bleu_score(spoiler, truth)
    tf_idf_spoilers.append((spoiler, truth))
    scorelist.append(score)

In [25]:
import numpy as np

In [26]:
np.mean(scorelist)

0.0

In [27]:
def calc_score_from_preds(preds, true_data):
    scorelist = []
    for idx, spoiler in enumerate(preds):
        truth = true_data[idx]
        score = calculate_bleu_score(spoiler, truth)
        scorelist.append(score)
    return scorelist

In [ ]:
calc_score_from_preds(preds, true_data)